<a href="https://colab.research.google.com/github/GuillermoBastian/LatamProject/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --------- Predicción de demora en vuelos - LATAM --------------

# 1. Importar Librerías

In [43]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
import pandas as pd 
import numpy as np

In [45]:
import seaborn as sns

# 2 Lectura & Añadir Campos Challenge

## 2.1 Load data

In [55]:
original_df = pd.read_csv('gdrive/My Drive/LATAM/dataset_SCL.csv', parse_dates=['Fecha-I', 'Fecha-O'])
feriados_chile = pd.read_excel("/content/gdrive/MyDrive/LATAM/Feriados_Chile.xlsx")
km_tpo = pd.read_excel("/content/gdrive/MyDrive/LATAM/KM_DISTANCIA.xlsx")
original_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


## 2.2 Borrar Variables que no agregan valor

In [56]:

cols_borrar = ["AÑO", "Ori-O", "Ori-I", "SIGLAORI"]

def delete_cols(df, cols_borrar):
  try:
    for k in cols_borrar:
      del df[k]
    print("OK: Columnas fueron borradas")
  except:
    print("ERROR: Columnas ya habían sido borradas")
    pass  

delete_cols(original_df, cols_borrar)

OK: Columnas fueron borradas


In [57]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami


## 2.4 Depurar missings!

In [58]:
original_df.isna().sum() > 0

Fecha-I      False
Vlo-I        False
Des-I        False
Emp-I        False
Fecha-O      False
Vlo-O         True
Des-O        False
Emp-O        False
DIA          False
MES          False
DIANOM       False
TIPOVUELO    False
OPERA        False
SIGLADES     False
dtype: bool

In [59]:
#Solo faltaba un dato del Nro. de vuelo, lo completaremos con el mismo código de Santiago a Lima con LAW para día jueves
original_df["Vlo-O"] = original_df["Vlo-O"].fillna(704)

## 2.3 Agregar Variables challenge

In [60]:
#1.- Calcular temporada_alta  -- si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
original_df["temporada_alta"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '12-15') | (original_df['Fecha-I'].dt.strftime('%m-%d') <= '03-03'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '07-15') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '07-31'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '09-11') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '09-30'), "temporada_alta"] = 1

#2.- Calcular dif_min 
original_df["dif_min"] = (original_df["Fecha-O"] - original_df["Fecha-I"])
original_df["dif_min"]  = original_df["dif_min"]/np.timedelta64(1,'m')

# 3.- Calcular atraso_15 
original_df["atraso_15"] = 0
original_df.loc[original_df["dif_min"] > 15, "atraso_15"] = 1

#4.- Calcular periodo_dia : mañana : 0 | tarde : 1 | noche : 2
original_df["periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '05:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '11:59'), "periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '12:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '18:59'), "periodo_dia"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '19:00') | (original_df['Fecha-I'].dt.strftime('%H:%M') <= '04:59'), "periodo_dia"] = 2


original_df.to_csv("synthetic_features.csv")

In [61]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami,1,3.0,0,2
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami,1,9.0,0,2
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami,1,9.0,0,2


## 2.4 Añadir Variables propias

In [62]:
feriados_chile.head(3)

,Evento,Mes,Dia
0,ano_nuevo,1.0,1.0
1,viernes_santo,4.0,14.0
2,sabado_santo,4.0,15.0


In [63]:
# 1.- Feriados:
main_nfields_df = pd.merge(original_df, feriados_chile,  how='left', left_on=['DIA','MES'], right_on = ['Dia','Mes'])
cols_borrar = ['Dia','Mes']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df['Evento'] = main_nfields_df['Evento'].fillna(0)

# 2.- Hora de partida
main_nfields_df["HORA"] = main_nfields_df["Fecha-I"].dt.hour + main_nfields_df["Fecha-I"].dt.minute/60

# 3.- Emp - Vuelo
main_nfields_df["Emp-Vlo"] = main_nfields_df["Emp-I"].astype(str) + main_nfields_df["Vlo-I"].astype(str)
main_nfields_df.head()

# 3.- Km diferencia + 4.- Tiempo de vuelo
main_nfields_df = pd.merge(main_nfields_df, km_tpo,  how='left', left_on=['SIGLADES'], right_on = ['SIGLADES'])
cols_borrar = ['Duración_hrs']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df

# 6.- Last_Day_Month


OK: Columnas fueron borradas
OK: Columnas fueron borradas


,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,...,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia,Evento,HORA,Emp-Vlo,KM,DURACION_MIN
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,...,Miami,1,3.0,0,2,ano_nuevo,23.500000,AAL226,6627,497.0
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,...,Miami,1,9.0,0,2,0,23.500000,AAL226,6627,497.0
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,...,Miami,1,9.0,0,2,0,23.500000,AAL226,6627,497.0
3,2017-01-04 23:30:00,226,KMIA,AAL,2017-01-04 23:33:00,226,KMIA,AAL,4,1,...,Miami,1,3.0,0,2,0,23.500000,AAL226,6627,497.0
4,2017-01-05 23:30:00,226,KMIA,AAL,2017-01-05 23:28:00,226,KMIA,AAL,5,1,...,Miami,1,-2.0,0,2,0,23.500000,AAL226,6627,497.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SPJC,JAT,2017-12-22 15:41:00,400.0,SPJC,JAT,22,12,...,Lima,1,46.0,1,1,0,14.916667,JAT400,2453,203.0
68202,2017-12-25 14:55:00,400,SPJC,JAT,2017-12-25 15:11:00,400.0,SPJC,JAT,25,12,...,Lima,1,16.0,1,1,navidad,14.916667,JAT400,2453,203.0
68203,2017-12-27 14:55:00,400,SPJC,JAT,2017-12-27 15:35:00,400.0,SPJC,JAT,27,12,...,Lima,1,40.0,1,1,0,14.916667,JAT400,2453,203.0
68204,2017-12-29 14:55:00,400,SPJC,JAT,2017-12-29 15:08:00,400.0,SPJC,JAT,29,12,...,Lima,1,13.0,0,1,0,14.916667,JAT400,2453,203.0


In [67]:
def calc_ult_dia (anio):
  lista_ult_dia = []
  lista_mes = []
  for i in range(1,13): 
    dummy_date = "{}-01-{}".format(str(i),str(anio))
    lista_mes.append(i)
    lista_ult_dia.append(pd.Period(dummy_date,freq='M').end_time.date().day)
  return lista_ult_dia 

calc_ult_dia(2017)

[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# 3 Análisis de datos

In [13]:
#Reconocer si la columna es categorica o numérica
def reconocer_cate_num(df, total_cols):
  numerical_columns = []
  categorical_columns = []

  for i in total_cols:
    if original_df[i].dtype == object:
      categorical_columns.append(i)
    else:
      numerical_columns.append(i)
  return categorical_columns, numerical_columns

total_cols = original_df.columns
categorical_columns, numerical_columns = reconocer_cate_num(original_df, total_cols)

original_df[categorical_columns] = original_df[categorical_columns].astype(str)

In [43]:
# Label Encoder
analysis_df = original_df

from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
categorical_df = oe.fit_transform(original_df[categorical_columns])
analysis_df[categorical_columns] = categorical_df

In [44]:
# Reconocer variables independientes del target
ycols = ["atraso_15"]

def get_complement_columns(total_cols, excluded_cols):
  xcols= []
  for i in total_cols:
    if i not in ycols:
      xcols.append(i)
  return xcols

xcols = get_complement_columns(total_cols, ycols)

In [39]:
original_df.describe()

,Vlo-I,Des-I,Emp-I,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
count,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000
mean,298.619110,36.238220,18.267909,444.317040,36.339882,21.051652,15.714790,6.622585,2.960135,0.541976,13.320470,23.700686,0.333724,9.110855,0.184940,0.882650
std,141.185192,13.966602,6.616948,223.423367,13.456680,6.451826,8.782886,3.523321,2.007787,0.498239,4.939165,17.599621,0.471546,19.313387,0.388252,0.782485
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-14.000000,0.000000,0.000000
25%,183.000000,31.000000,17.000000,255.000000,31.000000,19.000000,8.000000,3.000000,1.000000,0.000000,12.000000,8.000000,0.000000,-1.000000,0.000000,0.000000
50%,299.000000,38.000000,17.000000,443.000000,38.000000,22.000000,16.000000,7.000000,3.000000,1.000000,12.000000,24.000000,0.000000,4.000000,0.000000,1.000000
75%,417.000000,46.000000,27.000000,644.000000,46.000000,28.000000,23.000000,10.000000,5.000000,1.000000,15.000000,37.000000,1.000000,12.000000,0.000000,2.000000
max,583.000000,63.000000,29.000000,861.000000,62.000000,31.000000,31.000000,12.000000,6.000000,1.000000,22.000000,61.000000,1.000000,161.000000,1.000000,2.000000


In [41]:
analysis_df[xcols].corr()["dif_min"]

#Resultados - VARIABLES:
# MES
# TIPOVUELO
# Vlo-I / Vlo-O
# SIGLADES

Vlo-I             0.068428
Des-I             0.027725
Emp-I             0.028418
Vlo-O             0.067838
Des-O             0.028055
Emp-O             0.002289
DIA              -0.006706
MES               0.077280
DIANOM            0.020266
TIPOVUELO        -0.072733
OPERA             0.008068
SIGLADES          0.052567
temporada_alta    0.020899
dif_min           1.000000
periodo_dia       0.035625
Name: dif_min, dtype: float64

In [45]:
analysis_df

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,219.0,9.0,0.0,2017-01-01 23:33:00,318.0,10.0,2.0,1,1,0.0,0.0,5.0,34.0,1,3.0,0,2
1,2017-01-02 23:30:00,219.0,9.0,0.0,2017-01-02 23:39:00,318.0,10.0,2.0,2,1,2.0,0.0,5.0,34.0,1,9.0,0,2
2,2017-01-03 23:30:00,219.0,9.0,0.0,2017-01-03 23:39:00,318.0,10.0,2.0,3,1,3.0,0.0,5.0,34.0,1,9.0,0,2
3,2017-01-04 23:30:00,219.0,9.0,0.0,2017-01-04 23:33:00,318.0,10.0,2.0,4,1,4.0,0.0,5.0,34.0,1,3.0,0,2
4,2017-01-05 23:30:00,219.0,9.0,0.0,2017-01-05 23:28:00,318.0,10.0,2.0,5,1,1.0,0.0,5.0,34.0,1,-2.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,340.0,58.0,14.0,2017-12-22 15:41:00,515.0,57.0,16.0,22,12,6.0,0.0,14.0,28.0,1,46.0,1,1
68202,2017-12-25 14:55:00,340.0,58.0,14.0,2017-12-25 15:11:00,515.0,57.0,16.0,25,12,2.0,0.0,14.0,28.0,1,16.0,1,1
68203,2017-12-27 14:55:00,340.0,58.0,14.0,2017-12-27 15:35:00,515.0,57.0,16.0,27,12,4.0,0.0,14.0,28.0,1,40.0,1,1
68204,2017-12-29 14:55:00,340.0,58.0,14.0,2017-12-29 15:08:00,515.0,57.0,16.0,29,12,6.0,0.0,14.0,28.0,1,13.0,0,1


In [ ]:
]flights = flights.pivot("month", "year", "passengers")
ax = sns.heatmap(flights)

,Evento,Mes,Dia
0,ano_nuevo,1.0,1.0
1,viernes_santo,4.0,14.0
2,sabado_santo,4.0,15.0
3,dia_trabajador,5.0,1.0
4,gloria_naval,5.0,21.0
5,san_pedro,6.0,26.0
6,virg_carmen,7.0,16.0
7,asuncion,8.0,15.0
8,independencia1,9.0,18.0
9,independencia2,9.0,19.0
